### pCrunch Update

Jake Nunemaker

Last Updated: 02/08/2021

In [14]:
import os
from fnmatch import fnmatch

import numpy as np
import pandas as pd
import ruamel.yaml as ry

from pCrunch import LoadsAnalysis, PowerProduction
from pCrunch.io import load_FAST_out
from pCrunch.utility import save_yaml, get_windspeeds, convert_summary_stats

def valid_extension(fp):
    return any([fnmatch(fp, ext) for ext in ["*.outb", "*.out"]])

#### Project Directory

In [15]:
output_dir = "/Users/jnunemak/Projects/pCrunch/BAR10/rank_0/"
results_dir = os.path.join(output_dir, "results")
save_results = True

outfiles = [
    os.path.join(output_dir, f) for f in os.listdir(output_dir)
    if valid_extension(f)
]

print(f"Found {len(outfiles)} files.")

Found 48 files.


#### Interacting with output files

In [16]:
# The new framework provides an object oriented framework to interact with
# output files. The easiest way to use this is to use the 'load_FAST_out' function.

outputs = load_FAST_out(outfiles[:3])
outputs

In [17]:
# An instance of 'OpenFASTBinary' (or 'OpenFASTAscii' if applicable) is created.
# The instance stores the raw data but also provides many useful methods for
# interacting with the data:

# print(outputs[0].data)
# print(outputs[0].time)
# print(outputs[0].channels)
# print(outputs[0].maxima)
# print(outputs[0].stddevs)

# Individual channel time series can also be accessed with dict style indexing:
outputs[0]["Wind1VelX"]

array([12.742136  , 12.7732935 , 12.80445099, ...,  9.66378784,
        9.49098396,  9.31818104])

#### pCrunch Configuration

In [18]:
# Channel magnitudes are defined in a dict:
magnitude_channels = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Define channels (and their fatigue slopes) in a dict:
fatigue_channels = {
    "RootMc1": 10,
    "RootMc2": 10,
    "RootMc3": 10
}

# Define channels to save extreme data in a list:
channel_extremes = [
    "RotSpeed",
    "RotThrust",
    "RotTorq",
    "RootMc1",
    "RootMc2",
    "RootMc3",
]

#### Run pCrunch

In [20]:
# The API has changed and is in more of an object oriented framework.
la = LoadsAnalysis(
    outfiles[:5],                           # The primary input is a list of output files
    magnitude_channels=magnitude_channels,  # All of the following inputs are optional
    fatigue_channels=fatigue_channels,      # 
    extreme_channels=channel_extremes,      #
    trim_data=(0,),                         # If 'trim_data' is passed, all input files will
)                                           # be trimmed to (tmin, tmax(optional))

la.process_outputs(cores=4)                 # Once LoadsAnalysis is configured, process outputs with
                                            # `process_outputs`. `cores` is optional but will trigger parallel processing if configured

#### Outputs

In [21]:
# The summary stats per each file are here:
la.summary_stats

,Wind1VelX,Wind1VelY,Wind1VelZ,Azimuth,BldPitch1,BldPitch2,BldPitch3,LSSTipMys,LSSTipMzs,LSShftFys,...,RtAeroFyh,RtAeroFzh,BLFLAP1,BLFLAP2,BLFLAP3,GenPwr,GenTq,RootMc1,RootMc2,RootMc3
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
BAR10_0_IEC_13.outb,7.331778,-6.869149,-4.288182,0.000772,6.118227,6.118227,6.118227,-13111.493164,-12674.125977,-165.555588,...,-5.922870e+06,-4.352968e+06,-7508.041606,-7538.544022,-6856.646326,-1.709636e+05,0.000000,2.066127e+06,6.123670e+06,5.847567e+06
BAR10_0_IEC_44.outb,8.667914,-11.789858,-6.978124,0.005214,12.433221,12.433221,12.433221,-21819.613281,-25325.222656,-385.993225,...,-3.907670e+06,5.864540e+07,3108.495682,-4699.187800,-6618.763643,-1.633114e+05,0.000000,-4.242427e+06,2.036104e+06,3.784935e+06
BAR10_0_IEC_05.outb,2.856447,-3.947478,-2.561280,0.005236,-0.000711,-0.000711,-0.000711,-9256.007812,-6742.917969,-79.591148,...,-1.975618e+06,2.786134e+06,-3493.678004,4489.294780,1109.401172,-9.360385e+05,-9904.555098,-2.828652e+06,-1.116475e+06,-6.598650e+06
BAR10_0_IEC_29.outb,-1.641610,-7.189809,-5.436992,0.013818,-0.002101,-0.002101,-0.002101,-14932.917969,-13265.808594,-126.139671,...,4.020631e+06,2.204353e+07,-515.067738,7287.945109,-6500.772840,1.177895e+06,14297.999188,3.440799e+06,1.619509e+05,-1.402842e+06
BAR10_0_IEC_09.outb,5.212037,-5.217299,-3.432661,0.001330,1.648894,1.648894,1.648894,-13388.862305,-10368.466797,-105.768608,...,1.228290e+07,4.560042e+06,3974.905589,-3006.706357,-5555.861299,-1.939158e+05,189.144435,-2.989209e+06,-2.894136e+05,7.052371e+06


In [24]:
# These are indexable by channel, stat:
la.summary_stats["Wind1VelX"]

,min,max,std,mean,abs,integrated
BAR10_0_IEC_13.outb,7.331778,23.038691,2.653315,14.787289,23.038691,-3692.376680
BAR10_0_IEC_44.outb,8.667914,36.075912,4.272328,23.286647,36.075912,-1881.130931
BAR10_0_IEC_05.outb,2.856447,11.880972,1.555660,6.877674,11.880972,-2227.173018
BAR10_0_IEC_29.outb,-1.641610,17.482164,3.332014,7.058670,17.482164,-1966.393449
BAR10_0_IEC_09.outb,5.212037,17.023720,2.103185,10.822902,17.023720,-2934.333147


In [25]:
la.summary_stats[("Wind1VelX", 'min')]

BAR10_0_IEC_13.outb    7.331778
BAR10_0_IEC_44.outb    8.667914
BAR10_0_IEC_05.outb    2.856447
BAR10_0_IEC_29.outb   -1.641610
BAR10_0_IEC_09.outb    5.212037
Name: (Wind1VelX, min), dtype: float64

In [27]:
# Or by file
la.summary_stats.loc["BAR10_0_IEC_13.outb"]

Wind1VelX  min           7.331778e+00
Wind1VelY  min          -6.869149e+00
Wind1VelZ  min          -4.288182e+00
Azimuth    min           7.724802e-04
BldPitch1  min           6.118227e+00
                             ...     
GenPwr     integrated   -1.709636e+05
GenTq      integrated    0.000000e+00
RootMc1    integrated    2.066127e+06
RootMc2    integrated    6.123670e+06
RootMc3    integrated    5.847567e+06
Name: BAR10_0_IEC_13.outb, Length: 1104, dtype: float64

In [28]:
# Damage equivalent loads are found here:
la.DELs

,RootMc1,RootMc2,RootMc3
BAR10_0_IEC_13.outb,17155.972672,18123.882926,18975.048930
BAR10_0_IEC_44.outb,20769.760533,20173.288148,20931.456407
BAR10_0_IEC_05.outb,12965.959096,12840.962463,13141.212547
BAR10_0_IEC_29.outb,17868.450676,17151.796573,16319.655247
BAR10_0_IEC_09.outb,15761.531479,16148.632706,16634.678742


In [29]:
# Extreme events:
la.extreme_events

{'RotSpeed': [{'time': 685.0500000000001,
   'RotSpeed': 8.59638500213623,
   'RotThrust': 556.005126953125,
   'RotTorq': 6768.57666015625,
   'RootMc1': 14070.55579269342,
   'RootMc2': 17947.08235471186,
   'RootMc3': 17004.08210931144},
  {'time': 283.0,
   'RotSpeed': 9.073287963867188,
   'RotThrust': 630.1870727539062,
   'RotTorq': 6677.99755859375,
   'RootMc1': 14259.785796075768,
   'RootMc2': 12513.589342673744,
   'RootMc3': 13571.727943383383},
  {'time': 448.82,
   'RotSpeed': 7.822919845581055,
   'RotThrust': 1103.539794921875,
   'RotTorq': 6732.35986328125,
   'RootMc1': 31848.65546017486,
   'RootMc2': 22403.73546229115,
   'RootMc3': 24231.932214343855},
  {'time': 657.52,
   'RotSpeed': 8.274258613586426,
   'RotThrust': 837.1314697265625,
   'RotTorq': 6777.07470703125,
   'RootMc1': 28096.38032484325,
   'RootMc2': 19810.039937813377,
   'RootMc3': 16020.785348512087},
  {'time': 688.6,
   'RotSpeed': 8.304394721984863,
   'RotThrust': 874.20751953125,
   'RotTo